Bu yazıda kendi modelimizi eğtiriken nelere dikakt etmemmiz gerektiğini ve doğru eğitim için cfg dosyasının nasıl editlenmesi gerektiği gibi konuları konuşacağız. Ideal bir model için doğru formda eğitim çok önemlidir bu yüzden başlayalım.

# ADIM 1 - Verisetini hazırlama

- İlk olarak obje resimleri indirilecek veya toplanarak bir kalasör haline getirilmeli,
- LabelImg tool'u ile bu klasördeki tüm resimlere label atılmalı,
- Sonrasında klasörün içinin tamamı " images.zip " olarak zip haline getirilmelidir.

# ADIM 2 - Drive ve Collab Ayarlamalı
- Google Drive için
    - yolov3 isimli bir klasör oluşturun
    - zip haline getirdiğiniz images.zip dosyasını buray yükleyin
- Google Collab İçin
    -
    -

# ADIM 3 - Drive Dosyalarına Erişin

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# ADIM 4 - Darknet'i Klonla, Konfigüre et ve Derle

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Bu noktadan sonra makefile dosyası yani derleme dosyasını nesne sınfımıza göre yeniden ayarlayacağız. MakeFile dosyasında değiştirmemiz gerekn şeyler :
- GPU = 1
- CUDANN = 1
- OPENCV = 1

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
!make

# ADIM 5 - .cfg Dosyasını Konfigüre Et
Eğitim parametrelerine göre
- yolov3.cfg
- yolov3-tiny.cfg
dosyalarından kullanacağımızı elden geçirmemiz lazım. Bunun için bir kopyasını yerel bilgisyaramıza indirelim.
- İndirilen kopyada aşağıdaki satrıları değiştirmeliyiz
    - batch = 64
    - subdivision = 16
    - max_batches = num_of_classes * 2000
    - filters = (num_of_classes + 5) * 3
    - classes = num_of_classes

In [ ]:
# .cfg dosyasını kopyalıyorum
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Tek sınıflı bir model için
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

# ADIM 6 - Names Dosyası Oluşturma

In [ ]:
!echo -e "Obje İsmi\nObje İsmi\n . . ." > data/obj.names

# ADIM 7 - .data Dosyası Oluşturma

In [ ]:
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [ ]:
# .cfg ve .names dosyalarını Drive'a kayıt ediyorum
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [ ]:
!mkdir data/obj

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

# ADIM 8 - train.txt Dosyası Oluşturma

In [ ]:
import glob

images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

# ADIM 9 - pre-trained ağırlıkları İndir

yolov3-tiny için farklı ağırlıkların indirilmesi gerekmektedir. Bu dosya yolov3 320 içindir.

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

# ADIM 10 - Eğitime Başla

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Yukarıdaki kodu 4 parçaya ayırırsak eğer :
1. /darknet detector train :: Train komutu
2. data/obj.data :: Train için kullanılacak veriler
3. yolov3_training.cfg :: Train için kullanıalcak .cfg dosyası
4. darknet53.conv.74 :: Train için kullanıalcak ağırlıklar

Önemli bir not : Bazı durumlarda Google Collab ile olan bağınız kesilebilir ve işlem durabilir (sonlanır) böyle bir durumda en baştan başlamak yerine Google Drive'ınızda bulunan " yolov3_training_last.weights " dosyasını kullanarak en son kayıt edilmiş süreçten başlayabilirisniz. Bunun için yapmanız gereken tek şey :

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights

Bu işlemi yolov3-tiny için yapıyor olsaydık :

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

# Eğitilen Modeli Kullanmak İçin
1. Ağırlık Dosyalarını Google Drive üzerinden indirin
2. Konfigrasyon dosyasını Collab veya Drive üzerinden indirin
3. İsim Dosyasını Collab veya Drive Üzerinden indirin
4. Opencv 3.4.2 veya üzeri bir sürüm kullanın

## Ağırlıkların İndirilmesi
Ağırlık dosyanızı indirmek için Google Drive hesabınızdaki ilgili klasöre girdiğinizde karşınıza birden fazla ağırlık dosyası çıkması oldukça muhtemel. Tüm aırlık dosyaları üzerinden tercih etmeniz gereken dosya "yolov3_training_last.weights" dosyasıdır.

Olurda modelinizde over-fitting durumu oluştu (over-fitting durumu modelin eğtim için kullanılan veri serinde tespit yapabilmesi ama farklı resimlerde yapamaması durumudur. Modelin aynı resimler üzerinden çok tekrar yapıp ezber yapmasıdır) bunu çözmek için daha önceki safalarda kayıt alınmış ağırlıkları kullanabilrisiniz.